In [ ]:
# default_exp detect

In [ ]:
#hide
# """
# the weights and images are stored my Google Drive
# """
from google.colab import drive
drive.mount('/content/drive')

# hope you're already connected to the google drive!
%cd "/content/drive/MyDrive/Coding/ModelAssistedLabel/"

Mounted at /content/drive
/content/drive/MyDrive/Coding/ModelAssistedLabel


In [ ]:
#hide
%run "_Synch.ipynb"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Coding/ModelAssistedLabel
install nbdev: f
git commit -m: working on detector class
hooksn
diffsn
Converted 00_config.ipynb.
Converted 01_split.ipynb.
Converted 02_train.ipynb.
Converted 03_detect.ipynb.
Converted index.ipynb.
converting: /content/drive/My Drive/Coding/ModelAssistedLabel/03_detect.ipynb
converting: /content/drive/My Drive/Coding/ModelAssistedLabel/index.ipynb
converting: /content/drive/My Drive/Coding/ModelAssistedLabel/01_split.ipynb
converting: /content/drive/My Drive/Coding/ModelAssistedLabel/02_train.ipynb
converting /content/drive/My Drive/Coding/ModelAssistedLabel/index.ipynb to README.md
Executing: git config --local include.path ../.gitconfig
Success: hooks are installed and repo's .gitconfig is now trusted



# Detecting Objects
> wrapping `yolov5/detect.py`

In [ ]:
#export

import os, torch
os.chdir("yolov5")
from pathlib import Path
from utils.plots import plot_one_box
from utils.general import check_img_size, non_max_suppression
from utils.datasets import LoadStreams, LoadImages
from utils.general import scale_coords, xyxy2xywh
from utils.torch_utils import select_device
from models.experimental import attempt_load
import cv2, os, base64, json
os.chdir("..")

os.system("pip install pillow")

class Detector:
  """A wrapper for training loading saved YOLOv5 weights
  
  requirements:
    GPU enabled"""
  def __init__(self, weight_path, conf_threshold = .4, iou_threshold = .45, imgsz = 416, save_dir="save_dir", write_annotated_images_to_disk=False):
    """ Constructor. I pulled the default numeric values above directly from the
    detect.py file. I added the option to save model output to both images and 
    to txt files

    Args:
      weight_path: the path to which the saved weights are stored
      conf_threshold: lower bound on the acceptable level of uncertainty for a 
                      bounding box
      iou_threshold: IoU helps determine how overlapped two shapes are.
      imgsz: resolution of image to process (assumes square)
      save_dir: where to write annotated images
      write_annotated_images_to_disk: save human-friendly annotated images to disk 
    """
    
    self.weight_path = weight_path
    self.conf_threshold = conf_threshold
    self.iou_threshold = iou_threshold
    self.imgsz = imgsz
    self.device = select_device()
    self.model = attempt_load(self.weight_path, map_location=self.device)  # load FP32 model
    self.names = self.model.module.names if hasattr(self.model, 'module') else self.model.names
    self.imgsz = check_img_size(self.imgsz, s=self.model.stride.max())  # check img_size
    self.half = self.device.type != 'cuda'
    if self.half:
      self.model.half()  # to FP16

    self.write_annotated_images_to_disk = write_annotated_images_to_disk
    self.save_dir = save_dir

  def make_dir(dir):
    """makes a directory provided that the directiory doesn't already exist
    
    Args:
      dir: Directory to create a path towards
    """
    if not os.path.exists(dir):
      os.makedirs(dir)

  def process_image(self, source):
    """Runs on the model with pre-specified weights an input image. See original
    detect.py for more details

    Args:
      source: A string path to pre-specified weights for the model
      save_unscuffed: create copy of the pre-image

    Reurns:
      A JSON-serializable object encoding bounding box information
    """
    override = None
    if os.path.exists(self.save_dir):
      override = input(f"Save directory '{self.save_dir}' exists. \n 'Enter' to continue, anything else to cancel operation")

    if override is None or len(override) == 0:
      return self.__process_image__(source)

    assert False, "this code shouldn't run"


  #not saving images speed up processing dramatically
  def __process_image__(self, source):
    "helper for process_image"
    results = []
    img = torch.zeros((1, 3, self.imgsz, self.imgsz), device=self.device)  # init img
    _ = self.model(img.half() if self.half else img) if self.device.type != 'cpu' else None  # run once
    dataset = LoadImages(source, img_size=self.imgsz)

    if self.write_annotated_images_to_disk:
      save_dir = Path(self.save_dir)
      Detector.make_dir(save_dir)

    for path, img, im0s, vid_cap in dataset:
      tmp = {}
      img = torch.from_numpy(img).to(self.device)
      img = img.half() if self.half else img.float()  # uint8 to fp16/32
      img /= 255.0  # 0 - 255 to 0.0 - 1.0
      
      if img.ndimension() == 3:
        img = img.unsqueeze(0)

      pred = self.model(img, augment=False)[0]
      pred = non_max_suppression(pred, self.conf_threshold, self.iou_threshold, agnostic=False)
      
      for i, det in enumerate(pred):
        p, s, im0, frame = path, '', im0s, getattr(dataset, 'frame', 0)
        p = Path(p) 
        
        s += '%gx%g ' % img.shape[2:]  # print string
        gn = torch.tensor(im0.shape)[[1, 0, 1, 0]]  # normalization gain whwh

        if len(det):
          det[:, :4] = scale_coords(img.shape[2:], det[:, :4], im0.shape).round()
          for c in det[:, -1].unique():
              n = (det[:, -1] == c).sum()  # detections per class
              s += f'{n} {self.names[int(c)]}s, '  # add to string
          
          if self.write_annotated_images_to_disk:
            tmp["unscuffed"] = f"{save_dir}/unscuffed-{p.name}"
            cv2.imwrite(tmp["unscuffed"], im0)

          tmp["predictions"] = []
          for *xyxy, conf, cls in reversed(det):
            xywh = (xyxy2xywh(torch.tensor(xyxy).view(1, 4)) / gn).view(-1).tolist()  # normalized xywh
            line = (cls, *xywh, conf) # label format
            tmp["predictions"].append(('%g ' * len(line)).rstrip() % line)

            # save image with bboxes drawn on top
            if self.write_annotated_images_to_disk: 
              label = f'{self.names[int(cls)]} {conf:.2f}'
              plot_one_box(xyxy, im0, label=label, color=[0,0,200], line_thickness=5)
              tmp["labeled"] = f"{save_dir}/labeled-{p.name}"
              cv2.imwrite(tmp["labeled"], im0)

    return tmp

> Note: 

In [ ]:
d = Detector(weight_path="pre-trained weights/21-2-25 1k-digits YOLOv5-weights.pt")

Fusing layers... 


In [ ]:
ls "Image Repo/labeled/images/Final Roboflow Export (841)/images/"

In [ ]:
d.process_image("Image Repo/labeled/images/Final Roboflow Export (841)/images/")

In [ ]:
!nbdev_build_lib

Converted 00_config.ipynb.
Converted 01_split.ipynb.
Converted 02_train.ipynb.
Converted 03_detect.ipynb.
Converted index.ipynb.


In [ ]:
ls test/

In [ ]:
!ls "train/images"

ls: cannot access 'train/images': No such file or directory


In [ ]:
#export

from ModelAssistedLabel.core import Defaults
import pandas as pd
import re, os, IPython

class ImageProcessor:
  def __init__(self, with_weights, save_labeled_img=False, save_unscuffed=False, data_yaml = None):
    self.with_weights = with_weights
    self.d=Detector(self.with_weights, save_labeled_img=save_labeled_img, save_unscuffed=save_unscuffed)
    if data_yaml is None:
      data = Defaults().data_yaml.split("\n")[-1]
      labels = [re.sub("\D", "", x) for x in data.split(":")[1].split(",")]
    else:
      labels = data_yaml
    self.labels = labels

  def process_new_images(self, folder_path):
    dir = os.listdir(path)
    results = []
    for filename in dir:
      results.append({filename: self.d.process_image(path+filename)})
      IPython.display.clear_output()
    self.results=[self.parse_prediction_for_frame(x) for x in results]
    return len(results)

  def parse_prediction_for_frame(self, r):
    k = list(r.keys())[0]
    predictions = pd.DataFrame([x.split(" ") for x in r[k]["predictions"]], columns=["yaml class index", "xcent", "ycent", "wdith", "height", "confidence"])
    readable_class = predictions["yaml class index"].apply(lambda x: self.labels[int(x)])
    predictions.insert(0, "class label", readable_class)
    predictions["filename"] = k
    return predictions.sort_values(by="xcent")

In [ ]:
ip = ImageProcessor(with_weights="pre-trained YOLOv5 weights/21-2-25 1k-digits YOLOv5-weights.pt")

Fusing layers... 


In [ ]:
%%time
ip.process_new_images(folder_path="Image Repo/unlabeled/21-3-18 rowing 8-12 /")

CPU times: user 7.53 s, sys: 320 ms, total: 7.85 s
Wall time: 8.39 s


200

# resolving conflicts
> overlapping bounding boxes

In [ ]:
#export
import PIL

def yoloBox_to_standard(result):
  """
  The standard YOLOv5 coordinate format is normed to 1. Need to extract the 
  original's image width and height to convert to a standard cartesian plane.

  Args:
    result: a dictionary that includes both 
        * a key called "filename" that points to the original image
        * a key called "predictions" created when the image is parsed with the
          YOLOv5 model 

  Returns:
    Convert the predictions converted to a full-scale Cartesian coordinate system.
  """
  out = {}
  for k,v in result.items():
    out[k] = v

  PILim= PIL.Image.open(result["preimage location"])
  width, height = PILim.width, PILim.height

  out["predictions"] = []
  for prediction in result["predictions"]:
    bbox = prediction["bbox"]
    out["predictions"].append({
        "class": (int(prediction["class index"])+1)%10,
        "confidence": prediction["confidence"],
        "height": PILim.height*bbox["height"],
        "width": PILim.width*bbox["width"],
        "x": PILim.width*(bbox["xcent"] - bbox["width"]/2),
        "y": PILim.height*(bbox["ycent"] - bbox["height"]/2)
        })
  return out

In [ ]:
#export
def bb_intersection_over_union(boxA, boxB):
  "Source: https://www.pyimagesearch.com/2016/11/07/intersection-over-union-iou-for-object-detection/"
  # determine the (x, y)-coordinates of the intersection rectangle
  xA = max(boxA[0], boxB[0])
  yA = max(boxA[1], boxB[1])
  xB = min(boxA[2], boxB[2])
  yB = min(boxA[3], boxB[3])

  # compute the area of intersection rectangle
  interArea = (xB - xA) * (yB - yA)

  # compute the area of both the prediction and ground-truth
  # rectangles
  boxAArea = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
  boxBArea = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])

  # compute the intersection over union by taking the intersection
  # area and dividing it by the sum of prediction + ground-truth
  # areas - the interesection area
  iou = interArea / float(boxAArea + boxBArea - interArea)

  # return the intersection over union value
  return iou

In [ ]:
#export
def sorted_api(predictions):
  """sorts a list of prediction in ascending x order

  Args:
    an array of dictionaries containing bbox predictions

  Returns:
    the bounding boxes read off from left to right"""
  return sorted(predictions, key=lambda x: x['x'])

def digit_arr_to_string(predictions):
  """"Human-friendly interpretation of predictions"""
  return "".join(str(int(x['class'])%90) for x in sorted_api(predictions))


def corners(r):
  """ 
  Defines the relevante coordinate information

  Args: 
    r: converts a DataFrame row to a tuple
    
  Returns:
    Caretensian coordinate information (anchor in the lower left)
  """
  return (r['x'], r['y'], r['x'] + r['width'], r['y'] + r['height'])

def calculate_iou(frame, threshold = .4):
  """
  Determines the pair of bounding boxes that has the highest IoU over the :threshold:

  Args:
    frame: the output of either a YOLOv5 model call or an API call
    threshold: maximum percentage of overlap allowed between the bounding boxes
  """
  previous = None
  overlappers = None

  for i in range(len(sorted_api(frame))):
    row = sorted_api(frame)[i]
    row["index"] = i
    if previous is None:
      previous = row
      continue

    iou = bb_intersection_over_union(corners(previous), corners(row))
    if iou > threshold:
      tmp = {row["confidence"]: row, previous["confidence"]: previous}
      overlappers = {
        "iou": iou,
        "least confident": tmp[min(tmp.keys())]
      }

    previous = row
  return overlappers

> Note: the following functions work together to form the core of parsing bounding
box information.

> Warning: I assume that screens have no more than 4 objects in them.

If extra bounding boxesare detected, I find the pair of boxes that has the highest IoU (Intersection 
over Union). Of that pair, I identify the box that has the a lower associated
confidence and remove it.

If there is still an excess of bounding boxes but no boxes that are overlapping
in excess of the :threshold:, then bounding boxes with the most deviated y are 
removed

In [ ]:
def interpret_predictions(predictions_arr):
  while len(predictions_arr) > 0 and len(predictions_arr) != len(remove_worst(predictions_arr)):
    predictions_arr = remove_worst(predictions_arr)
  return predictions_arr

def remove_worst_offender(res, threshold):
  overlappers = calculate_iou(res)
  
  if overlappers is not None:
    included = [x for x in res if x["index"] != overlappers["least confident"]["index"]]
    return included
  else: #look for y outlier
    df = pd.DataFrame.from_dict(res)
    # print(df.sort_values(by="confidence",ascending=False))
    df["zscore"] = abs(df.y - df.y.mean())/df.y.std() #FIXME
    return df[df["zscore"] < 1.5].to_dict('records')

def remove_worst(res, threshold=.4):
  return remove_worst_offender(res, threshold)

### post-processing

In [ ]:
#export

import io, PIL, base64

def raw_parse_from_json(json_elements, loaded_detector):
  """
  Helper to interpret raw base 64 images. Creates a new JPG file from the base64
  data and feeds the image to the Detector

  Args:
    json_elements: array of dictionaries from file. Assert existence of a "base64"
      key where the images are stored
    loaded_detector: a valid Detector
  
  returns:
    the result of processessing all of the base64 images
  """
  out = []
  for idx in range(len(json_elements)):
    from64name = Defaults._itername("(", ") image.jpg")
    b64image = json_elements[idx]["base64"]
    im = PIL.Image.open(io.BytesIO(base64.b64decode(b64image)))
    im.save(from64name)
    
    single = loaded_detector.process_image(from64name)
    single["preimage location"] = from64name
    out.append(single)
  return out

def parse_from_json(json_elements, loaded_detector):
  """
  Gets a JSON file containing the base64 images of data to label pushed through
  a Detector. Post-processing automatically de-norms the YOLOv5 bounding boxes 
  based on the original image's height and width

  Args:
    json_elements: array of dictionaries from file. Assert existence of a "base64"
      key where the images are stored
    loaded_detector: a valid Detector
  
  returns:
    the bounding boxes associated with each image
  """
  preprocessed = raw_parse_from_json(json_elements, loaded_detector)
  ds = DataSet(preprocessed)
  processed=ds.export()

  for i in range(len(processed)):
    processed[i]["preimage location"] = processed[i]["filename"]

  postprocess = []
  for x in processed:
    postprocess.append(yoloBox_to_standard(x))
  return postprocess